## Consultar los trabajos de GetOnBoard con Python3

El siguiente programa utiliza los datos de trabajos de programación de la página GetOnBoard para mostrarlos en un DataFrame, filtrarlos y luego graficar los resultados filtrados.

## Preparativos
Primero, instalamos e importamos las bibliotecas necesarias: pandas para manejar los datos en un DataFrame, requests para realizar llamadas a la API, re para manipular texto y matplotlib para graficar.
```
! pip install pandas,requests,matplotlib

import pandas as pd
import requests
import re
import matplotlib.pyplot
```

Luego, definimos una función para limpiar los datos que contengan etiquetas HTML.


```
def remove_html_tags(text):
    clean_text = re.sub(r'<[^>]*>', ' ', str(text))
    return clean_text
```

Finalmente, definimos una función para filtrar trabajos por palabras clave.


```
def filter_jobs_by_keyword(df, keywords):
    filtered_df = pd.DataFrame()
    for keyword in keywords:
            filtered_df = pd.concat([filtered_df, df[df['desirable'].str.contains(keyword, case=False, na=False)]])
    return filtered_df

```

## Flujo de código

A continuación se mostrará el código del programa explicando brevemente lo que realiza.

## Llamada a la API y normalización de datos

Primero, realizamos una llamada a la API de GetOnBoard y normalizamos los datos con pandas.


```
response = requests.get('https://www.getonbrd.com/api/v0/categories/programming/jobs')
responseInJSON = response.json();
data = responseInJSON['data'];
df = pd.json_normalize(data);

```

Es importante mencionar que el metodo .json_normalize ya crea un dataframe normalizado.

## Limpieza de datos

Los datos de "functions" y "derisable" que entrega esta API están sucios con etiquetas HTML dentro del JSON, por lo tanto, para una mejor lectura es mejor limpiar esos datos. Aquí utilizamos la función antes definida para limpiar etiquetas HTML.

```
df['attributes.functions'] = df['attributes.functions'].apply(remove_html_tags);
df['attributes.desirable'] = df['attributes.desirable'].apply(remove_html_tags);
```
## Construir el Dataframe

Primero definiremos un arreglo con las columnas que necesitamos para el muestreo de datos, en este caso se eligen las siguientes.

```
columns_to_select = ['attributes.title', 'attributes.functions', 'attributes.desirable', 
                     'attributes.min_salary', 'attributes.max_salary', 'attributes.modality']
```
Luego, se verifica si es que la columna de "modality" es nula o no, debido a que varios trabajos que entrega la API no presentan este dato. Si se encuentra un dato nulo en modality, se elimina la columna. (puedes reemplazar este comportamiento por lo que tu quieras). Y finalmente se imprime el dataframe creado.

```
if 'attributes.modality' in df.columns:
    finalDf= df[columns_to_select]
    finalDf = df[[col for col in columns_to_select]]
    finalDf.columns=['title', 'functions', 'desirable', 'min_salary', 'max_salary']
else:
    columns_to_select.remove('attributes.modality')
    finalDf = df[[col for col in columns_to_select]]
    finalDf.columns=['title', 'functions', 'desirable', 'min_salary', 'max_salary']
    
finalDf
```
Con esto ya tenemos nuestro primer dataframe mostrando todos los trabajos con las columnas que quisimos mostrar.

## Filtrado por palabra

Primero, eliminaremos cualquier fila que tenga como "max_salary" NaN, ya que, en este caso nos interesa saber cuanto es el salario máximo que puede ganar un programador en base a las funciones que deba ejercer.

```
cleaned_df = finalDf.dropna(subset=['min_salary', 'max_salary'])

```

Luego, utilizamos la funcion filter_jobs_by_keyword() creada anteriormente para filtrar por palabras.

```
filtered_jobs = filter_jobs_by_keyword(cleaned_df,["Scrum","Microservicios","MongoDB"," .NET ", "Java"]);
filtered_jobs
```

## Gráfico

Para finalizar, utilizamos matplotlib.pyplot para graficar los datos filtrados.

El parámetro figsize especifica las dimensiones de la figura en pulgadas (ancho, alto). En este caso, (10, 6) indica que la figura tendrá 10 pulgadas de ancho y 6 pulgadas de alto.

```
plt.figure(figsize=(10, 6))
```

Luego, le indicamos al gráfico de barras quien será su eje x y quien será su eje Y. Se rota el label para los elementos del eje x en 45 grados para una mejor lectura y visibilidad, y finalmente se muestra el gráfico.

```
plt.bar(filtered_jobs['title'], filtered_jobs['max_salary'], color='skyblue')
plt.xticks(rotation=45, ha='right')
plt.show()
```



In [ ]:
%pip install pandas
%pip install requests
%pip install matplotlib

In [ ]:
import sys
print(sys.executable)


In [ ]:
import pandas as pd
import re
import requests
import matplotlib.pyplot as plt



In [ ]:
def remove_html_tags(text):
    clean_text = re.sub(r'<[^>]*>', ' ', str(text))
    return clean_text

In [ ]:
response = requests.get('https://www.getonbrd.com/api/v0/categories/programming/jobs')
responseInJSON = response.json();
data = responseInJSON['data'];
df = pd.json_normalize(data);


In [ ]:
df['attributes.functions'] = df['attributes.functions'].apply(remove_html_tags);
df['attributes.desirable'] = df['attributes.desirable'].apply(remove_html_tags);


In [ ]:

columns_to_select = ['attributes.title', 'attributes.functions', 'attributes.desirable', 
                     'attributes.min_salary', 'attributes.max_salary', 'attributes.modality']

if 'attributes.modality' in df.columns:
    finalDf= df[columns_to_select]
    finalDf = df[[col for col in columns_to_select]]
    finalDf.columns=['title', 'functions', 'desirable', 'min_salary', 'max_salary']
else:
    columns_to_select.remove('attributes.modality')
    finalDf = df[[col for col in columns_to_select]]
    finalDf.columns=['title', 'functions', 'desirable', 'min_salary', 'max_salary']
    
finalDf

In [ ]:
def filter_jobs_by_keyword(df, keywords, column):
    filtered_df = pd.DataFrame()
    for keyword in keywords:
            filtered_df = pd.concat([filtered_df, df[df[column].str.contains(keyword, case=False, na=False)]])
    return filtered_df

In [ ]:
request = requests.get("https://www.getonbrd.com/api/v0/categories?per_page=100&page=1");
dataJson = request.json();
data = dataJson['data'];
categories = []
for element in data:
    categories.append(element['id']);
categories



In [ ]:
len(categories)

In [ ]:
response2 = []
for cat in categories:
    request2 = (requests.get(f'https://www.getonbrd.com/api/v0/categories/{cat}/jobs?expand=["tags"]'))
    response2.append(request2.json()['data'])
response2

In [ ]:
ultimateDf = pd.DataFrame(response2)
ultimateDf

In [ ]:

ultimateDf.to_csv( 'trabajitos' , index=False);

In [ ]:
jobs = []
for i in range(len(categories)):
    for j in range(len(response2[i])):
        jobs.append(response2[i][j])
jobs

In [ ]:
jobsDf = pd.json_normalize(jobs)
jobsDf = jobsDf.drop(['type','id','attributes.projects', 'attributes.description_headline', 'attributes.location_tenants.data', 'attributes.location_cities.data'], axis=1) # Drop innecesary columns, sacar headlines y campos sin información
jobsDf['attributes.functions'] = jobsDf['attributes.functions'].apply(remove_html_tags);
jobsDf['attributes.benefits'] = jobsDf['attributes.benefits'].apply(remove_html_tags);
jobsDf['attributes.description'] = jobsDf['attributes.description'].apply(remove_html_tags);
jobsDf

In [ ]:
print(jobsDf['attributes.tags.data'])

# Información General
1. ¿Cuántas categorías de trabajos hay en el dataset?

In [ ]:
print("Existen",len(categories), "categorias de trabajos en el dataset")

2.¿Cuántos trabajos hay en total en el dataset?

In [ ]:
print("Hay un total de", jobsDf['attributes.title'].size, "trabajos en el dataset" )

3.¿Cuál es la categoría con más ofertas de trabajo?

In [ ]:
#.sort_values(ascending=False) ordena los valores de esta columna en orden descendente (de mayor a menor).
# .index devuelve los índices de los valores ordenados
aplicationsSort = jobsDf['attributes.applications_count'].sort_values(ascending=False).index
print("El trabajo con más aplicaciones es", jobsDf['attributes.title'][aplicationsSort[0]], "con", jobsDf['attributes.applications_count'][aplicationsSort[0]], "aplicaciones")

4.¿Cuál es la categoría con menos ofertas de trabajo?

In [ ]:
#.sort_values(ascending=True) ordena los valores de esta columna en orden ascendente (de menor a mayor)., en este caso se revisó
#que el trabajo con menos aplicaciones es 0 por lo que se buscaron todos los trabajos con este numero de aplicaciones en vez de utilizar .sort_values(ascending=True)
# .index devuelve los índices de los valores ordenados
jobsWithouAplications = jobsDf[jobsDf['attributes.applications_count'] == 0] # Se filtran los trabajos con 0 aplicaciones :D
print("Hay", jobsWithouAplications['attributes.title'].size, "trabajos sin aplicaciones")
print("La lista de trabajos sin aplicaciones es:"," \n " ,jobsWithouAplications['attributes.title'].values,"\n"
      "con un total de", jobsWithouAplications['attributes.applications_count'].values[0], "aplicaciones cada uno")

# Detalles por Trabajo
5. ¿Cuáles son los títulos de todos los trabajos en la categoría de 'Programming'?

In [ ]:
# Se filtran los trabajos que contienen las palabras 'Programador', 'Desarrollador' o 'Developer' en su título, 
#case=False para que no sea sensible a mayúsculas y minúsculas, na=False para que no tome en cuenta los valores nulos
programingTitles = jobsDf[jobsDf['attributes.title'].str.contains("Programador|Desarrollador|Developer|Programming", case=False, na=False)] 
print("Hay", programingTitles['attributes.title'].size, "trabajos que contienen las palabras 'Programador', 'Desarrollador', 'Developer' o 'Programming' en su título")

6.¿Qué porcentaje de trabajos requieren experiencia en 'Remote' o 'Hybrid' en comparación con los trabajos presenciales?

In [ ]:
remoteJobs = jobsDf[jobsDf['attributes.remote'] == True] # Se filtran los trabajos que son remotos
print("Hay", remoteJobs['attributes.title'].size, "trabajos remotos")
hybridJobs = jobsDf[jobsDf['attributes.remote_modality'] == 'hybrid'] # Se filtran los trabajos que son híbridos
print("Hay", hybridJobs['attributes.title'].size, "trabajos híbridos")
presentialJobs = jobsDf[(jobsDf['attributes.remote'] == False) & (jobsDf['attributes.remote_modality'] != 'hybrid')] # Se filtran los trabajos que son presenciales
print("Hay", presentialJobs['attributes.title'].size, "trabajos presenciales")

totalRemoteHybridJobs = remoteJobs['attributes.title'].size + hybridJobs['attributes.title'].size
presentialJobs = presentialJobs['attributes.title'].size

print("Hay", totalRemoteHybridJobs, "trabajos que requieren experiencia en remoto o híbrido")
if presentialJobs > 0:  # Evitar división por cero pofavo
    porcentajeRH = (totalRemoteHybridJobs/(presentialJobs+totalRemoteHybridJobs)) * 100;
    porcentajeP = (presentialJobs/(presentialJobs+totalRemoteHybridJobs))*100;
    print("El porcentaje de trabajos que requieren experiencia en remoto o híbrido es ", porcentajeRH,"%");
    print("El porcentaje de trabajos que requieren experiencia en presencial es ", porcentajeP,"%");
else:
    print("No hay trabajos presenciales para comparar.")

7.¿Cuáles son las tres habilidades más comúnmente solicitadas en la categoría de 'Data Science / Analytics'?

In [ ]:
dfDataScienceAnalytics = filter_jobs_by_keyword(jobsDf,['Data Science / Analytics'],'attributes.category_name')
dfDataScienceAnalytics


In [ ]:
from collections import Counter;
tag_counter = Counter()

# Iterar sobre cada fila del dataframe
for _, row in dfDataScienceAnalytics.iterrows():
    tags = row["attributes.tags.data"]
    for tag in tags:
        tag_id = tag['id']
        tag_counter[tag_id] += 1

# Obtener los 3 tags más frecuentes
most_common_tags = tag_counter.most_common(3)

# Mostrar los resultados
print(most_common_tags)

In [ ]:
dfCert = filter_jobs_by_keyword(jobsDf,['certifi'],'attributes.description');
dfCert

In [ ]:
dfCert.shape

## 46 registros que solicitan certificaciones

## Pregunta 4

In [ ]:
tag_counter = 0

# Iterar sobre cada fila del dataframe
for _, row in jobsDf.iterrows():
    tags = row["attributes.tags.data"]
    for tag in tags:
        tag_id = tag['id']
        if (tag_id == "english"):
            tag_counter += 1;

print((tag_counter/jobsDf.shape[0])*100, "%")

In [ ]:
dfExp = filter_jobs_by_keyword(jobsDf,['experiencia previa'],'attributes.description');
dfExp2 = filter_jobs_by_keyword(jobsDf,['experiencia previa'],'attributes.desirable');

dfExp.shape[0]+dfExp2.shape[0]


In [ ]:
categories

In [ ]:
dfUX = filter_jobs_by_keyword(jobsDf,['Design / UX'],'attributes.category_name')
dfUX

In [ ]:
jobsDf

In [ ]:
print(jobsDf['attributes.tags.data'].tolist()[0])

In [109]:
tagsArray = []
nan = float('nan')
for _, row in jobsDf.iterrows():
    tagInJob = []
    tags = row["attributes.tags.data"]
    salary = ((row['attributes.max_salary']+row['attributes.min_salary'])/2)
    pais = row['attributes.countries']
    binary = "mayor que mil" if salary>1500 else "menor que 1500"
    title = row["attributes.title"]
    if tags and pd.notna(salary):
        for tag in tags:
            tagInJob.append(tag['id'])
        tagsArray.append(tagInJob)
        tagInJob.append(binary)
        tagInJob.append(title)
        tagInJob.append(pais[0])
        
tagsArray

[['excel',
  'customer-service',
  'crm',
  'hubspot',
  'customer-success',
  'menor que 1500',
  'Agente de Customer Success',
  'Remote'],
 ['excel', 'menor que 1500', 'Customer Implementation Specialist', 'Remote'],
 ['excel',
  'customer-service',
  'english',
  'customer-success',
  'menor que 1500',
  'Customer Success Executive',
  'Chile'],
 ['javascript',
  'python',
  'agile',
  'front-end',
  'angularjs',
  'virtualization',
  'react',
  'amazon-web-services',
  'docker',
  'kubernetes',
  'cloud-computing',
  'back-end',
  'ci-cd',
  'cloud',
  'angular',
  'menor que 1500',
  'Programador Junior Soporte de Soluciones Cloud',
  'Peru'],
 ['onboarding-specialist',
  'menor que 1500',
  'Onboarding Specialist Mexico',
  'Remote'],
 ['saas',
  'customer-success',
  'mayor que mil',
  'Customer Success Manager México',
  'Remote'],
 ['customer-success',
  'menor que 1500',
  'Customer Success Onboarding',
  'Chile'],
 ['excel', 'menor que 1500', 'Fleet Assistant (Soporte Terre

In [ ]:
perksArray = []
for _, row in jobsDf.iterrows():
    perksInJob = []